In [1]:
import tensorflow as tf 
import numpy as np
import sys
from layers import FcLayerDeploy, ConvLayerDeploy

Matplotlib is building the font cache; this may take a moment.


In [2]:
def load_weights(directory, name):
	
	weights = np.load(directory + '/' + name + '-weights.npy')
	prune_mask = np.load(directory + '/' + name + '-prune-mask.npy')

	return weights, prune_mask

In [3]:
if __name__ == "__main__":
    
    weights_dir = './weights'
    
    x_PH = tf.placeholder(tf.float32, [None, 28, 28, 1])
    print(x_PH)
    weights, prune_mask = load_weights(weights_dir, 'conv1')

Tensor("Placeholder:0", shape=(?, 28, 28, 1), dtype=float32)


In [4]:
	L1 = ConvLayerDeploy(weights, prune_mask, x_PH.shape[1], x_PH.shape[2], 2, 'conv1')

layer: conv1
	valid matrix weights: 800
	total tensor weights: 800
	total matrix weights: 5274752


In [5]:
    x = L1.forward_matmul_preprocess(x_PH)
    print("First x",x)
    x = tf.nn.relu(L1.forward_matmul(x))
    print("Second x",x)
    x = L1.forward_matmul_postprocess(x)
    print("Third x",x)

First x Tensor("Reshape:0", shape=(?, 784), dtype=float32)

Second x Tensor("Relu:0", shape=(?, 6728), dtype=float32)
Third x Tensor("transpose_3:0", shape=(?, 14, 14, 32), dtype=float32)


In [74]:
    weights, prune_mask = load_weights(weights_dir, 'conv2')
    L2 = ConvLayerDeploy(weights, prune_mask, x.shape[1], x.shape[2], 2, 'conv2')
    x = L2.forward_matmul_preprocess(x)
    print("First x",x)
    x = tf.nn.relu(L2.forward_matmul(x))
    print("Second x",x)
    x = L2.forward_matmul_postprocess(x)
    print("Third x",x)

layer: conv2
	valid matrix weights: 51200
	total tensor weights: 51200
	total matrix weights: 22579200
First x Tensor("Reshape_27:0", shape=(?, 6272), dtype=float32)
Second x Tensor("Relu_15:0", shape=(?, 3600), dtype=float32)
64
Printing x Tensor("Reshape_28:0", shape=(?, 64, 7, 7), dtype=float32)
Third x Tensor("transpose_61:0", shape=(?, 7, 7, 64), dtype=float32)


In [6]:
    x = tf.reshape(x, [-1, 7 * 7 * 64])
    print("Reshaped x - Expected 3136 -->",x)

Reshaped x - Expected 3136 --> Tensor("Reshape_2:0", shape=(?, 3136), dtype=float32)


In [7]:
    weights, prune_mask = load_weights(weights_dir, 'fc1')
    L3 = FcLayerDeploy(weights, prune_mask, 'fc1')	
    x = tf.nn.relu(L3.forward_matmul(x))
    print("Printing FC layer x",x)

layer: fc1
	valid matrix weights: 3211264.0
	total matrix weights: 3211264
Printing FC layer x Tensor("Relu_1:0", shape=(?, 1024), dtype=float32)


In [8]:
    weights, prune_mask = load_weights(weights_dir, 'fc2')
    L4 = FcLayerDeploy(weights, prune_mask, 'fc2')	
    logits = L4.forward_matmul(x)
    print("Printing logits or FC layer x -->", logits)

layer: fc2
	valid matrix weights: 10240.0
	total matrix weights: 10240
Printing logits or FC layer x --> Tensor("transpose_7:0", shape=(?, 10), dtype=float32)


In [9]:
	labels = tf.placeholder(tf.float32, [None, 10])

In [10]:
	correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels, 1))

In [11]:
print(correct_prediction)

Tensor("Equal:0", shape=(?,), dtype=bool)


In [12]:
	accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))	

In [86]:
print(accuracy)

Tensor("Mean_2:0", shape=(), dtype=float32)


In [13]:
	from tensorflow.examples.tutorials.mnist import input_data
	mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
	
	sess = tf.Session()
	
	batches_acc = []

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [14]:
    for i in range(10):

        batch_x, batch_y = mnist.test.next_batch(1000)
        batch_x = np.reshape(batch_x,(-1, 28, 28, 1))	
        #print(labels)
        batch_acc = sess.run(accuracy,feed_dict={x_PH: batch_x, labels: batch_y})
		#batches_acc.append(batch_acc)
				
	#acc = np.mean(batches_acc)
		


InvalidArgumentError: k (784) from index[4352,1] out of bounds (>=784)
	 [[node SparseTensorDenseMatMul/SparseTensorDenseMatMul (defined at /home/mohan/anaconda2/envs/mlp8/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'SparseTensorDenseMatMul/SparseTensorDenseMatMul':
  File "/home/mohan/anaconda2/envs/mlp8/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mohan/anaconda2/envs/mlp8/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mohan/.local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mohan/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/mohan/.local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 612, in start
    self.io_loop.start()
  File "/home/mohan/.local/lib/python3.6/site-p

In [32]:
	print('deploy accuracy:', acc)

NameError: name 'acc' is not defined